In [ ]:
import optuna
import pandas as pd
import numpy as np
import yaml
import pickle
import torch
import os

from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch import tensor, cat, save, load, optim, nn
from torch.utils.data import DataLoader

from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend

import src.study_handler as sh
import src.save_load as sl
from src.utils import print_yaml
import multiprocessing
from src.study_handler import run_baseline_optimization
from src.dataset_handler import processDataset, loadDataset

In [ ]:
#---------------#
#  Open Config  #
#---------------#
config = None
with open("./study.yaml") as file:
    config = yaml.safe_load(file)
    
print_yaml(config)

In [ ]:
#-------------------#
#  Prepare dataset  #
#-------------------#
data_cfg = config['data']

trainset, testset, full_dataset = loadDataset(data_cfg)
if full_dataset is None:
    train_dataset, test_dataset, train_indices, test_indices = processDataset(data_cfg, trainset, testset, None, None)
else:
    train_dataset, test_dataset, train_indices, test_indices = processDataset(data_cfg, None, None, None, full_dataset)

# Assign the datasets
sh.train_dataset = train_dataset
sh.test_dataset = test_dataset
print(f"train_dataset data shape: {sh.train_dataset.dataset.data.shape}")
print(f"train_dataset targets shape: {sh.train_dataset.dataset.targets.shape}")

In [ ]:
# ------------------------#
#        Run study        #
# ------------------------#
gpu_ids = [0, 1, 2, 3, 4, 5, 6]

def parallell_optimization(config, gpu_ids):
    study_cfg = config['study'] 
    print(f"Starting parallell optimization using the following gpu ids: {gpu_ids}")

    metadata = sl.buildStudyMetadata(study_cfg, config['data']) 
    hash_id, save_path = sl.saveStudy(metadata, savePath=study_cfg['root'])
    
        
    metadata = sl.buildStudyMetadata(study_cfg, config['data'])
    hash_id, save_path = sl.saveStudy(metadata, savePath=config['study']['root'])
    
    # split up the trials among the gpus
    total_trials = study_cfg["trials"]
    n_gpus = len(gpu_ids)
    base_trials, remainder = divmod(total_trials, n_gpus)
    
    # assign trials per GPU
    trials_per_gpu = [base_trials + 1 if i < remainder else base_trials for i in range(n_gpus)]

    processes = []
    for gpu_id, trials in zip(gpu_ids, trials_per_gpu):
        print(f"Running {trials} on gpu: {gpu_id}")
        p = multiprocessing.Process(
            target=run_baseline_optimization,
            args=(config, gpu_id, trials, save_path, hash_id)
        )
        processes.append(p)
    
    for p in processes:
        p.start() 
    for p in processes:
        p.join()
        
    db_path = os.path.join(study_cfg['root'], "baseline_study.db")
    storage = f"sqlite:///{db_path}"
    study_name = f"{study_cfg['study_name']}-{hash_id}"
    study = optuna.load_study(study_name=study_name, storage=storage)
    return study

if config is not None:
    #run_optimization(config)
    parallell_optimization(config, gpu_ids)